# Fake job Prediction: Investigating the pattern to avoid the job scam

# Notebook 5  Neural network and Modeling

**Fifth setp: Modeling with Pre-trained word vectors and Neural network**

By: Polly Pang

In this portion of the notebook, I will use Pre-trained word vectors, convert text column into sparse metrics. And implement traditional Ml models and also Neural network model.

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home"> Contents</h3>

* [1. Libraries](#1)
* [2. Load data](#2)
* [3. NLP](#3)
    - [3.1 Pre-process](#3.1)
* [4. Word Embeddings](#4)
    - [4.1 Word2Vec](#4.1)
    - [4.2 GloVe](#4.2)
* [5. Modeling](#5)
    - [5.1 oversampling](#5.1)

    
    
    
* [7. End of Notebook 5](#7)

# 1. Libraries <a id="1"></a>

In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import joblib
from sklearn.model_selection import train_test_split

# NLP
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from tqdm import tqdm

# 2. Load data

## 2.1 Target and Feature<a id="2.1"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

**plan**
- Define X and y, Target :`fraudulent`, Features:`has_salary_range`,`telecommuting`,`has_company_logo`,`has_questions`,and `text`.
- split data into test set and reminder set, as I will process Cross-Validation in the follwing part, so I will not split my dataset into train and test.
- investigate the distribustion of reminder set.

In [154]:
# load data
word_embed_df=joblib.load('data/df_model_full_df.pkl')

In [155]:
# sample of dataset
word_embed_df

,has_salary_range,telecommuting,has_company_logo,has_questions,fraudulent,text
0,0,0,1,0,0,"Marketing Intern Marketing We're Food52, and w..."
1,0,0,1,0,0,Commissioning Machinery Assistant (CMA) Blank ...
2,0,0,1,0,0,Account Executive - Washington DC Sales Our pa...
3,0,0,1,1,0,Bill Review Manager Blank SpotSource Solutions...
4,0,0,0,0,0,Accounting Clerk Blank Blank Job OverviewApex ...
...,...,...,...,...,...,...
17874,0,0,1,1,0,Account Director - Distribution Sales Vend is ...
17875,0,0,1,1,0,Payroll Accountant Accounting WebLinc is the e...
17876,0,0,0,0,0,Project Cost Control Staff Engineer - Cost Con...
17877,0,0,0,1,0,Graphic Designer Blank Blank Nemsia Studios is...


In [156]:
print(f"The shape of word_embed_df is {word_embed_df.shape}")

The shape of word_embed_df is (17879, 6)


In [157]:
word_embed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17879 entries, 0 to 17878
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   has_salary_range  17879 non-null  int32 
 1   telecommuting     17879 non-null  int64 
 2   has_company_logo  17879 non-null  int64 
 3   has_questions     17879 non-null  int64 
 4   fraudulent        17879 non-null  int64 
 5   text              17879 non-null  object
dtypes: int32(1), int64(4), object(1)
memory usage: 768.4+ KB


**Define Target and Features**

In [158]:
# define X and y
X=word_embed_df.drop('fraudulent',axis=1)
y=word_embed_df['fraudulent']

In [159]:
print(f"The shape of features(X) is {X.shape},and the shape of y (Target) is {y.shape}")

The shape of features(X) is (17879, 5),and the shape of y (Target) is (17879,)


**Reminder and test split**

In [160]:
X_rem,X_test,y_rem,y_test=train_test_split(X,y,test_size=0.3,random_state=5, stratify=y)

In [161]:
print(f"The shape of X_reminder is {X_rem.shape},and the shape of y_rem is {y_rem.shape}")

The shape of X_reminder is (12515, 5),and the shape of y_rem is (12515,)


In [162]:
print(f"The shape of X_test is {X_test.shape},and the shape of y_test is {y_test.shape}")

The shape of X_test is (5364, 5),and the shape of y_test is (5364,)


In [163]:
X_rem.head()

,has_salary_range,telecommuting,has_company_logo,has_questions,text
4310,0,0,1,1,Creative Director - Art Blank Kettle is an ind...
11082,0,0,1,0,"CDL Driver Blank ABC Supply Co., Inc. is the n..."
14188,0,0,1,0,Data Analyst (Marketing) Blank Blank We are lo...
8280,0,0,1,1,Channel Representative Blank Intercom (# is a ...
13924,0,0,1,0,Programmatic Media Manager Media Since 1978Our...


In [164]:
y_rem.unique()

array([0, 1], dtype=int64)

In [165]:
X_test.head()

,has_salary_range,telecommuting,has_company_logo,has_questions,text
8222,1,0,1,0,eCommerce Specialist / Database Management for...
1560,0,0,1,1,Solution Engineer Blank Declara is focused on ...
17171,1,0,0,1,Sales Representative for Home Improvement Blan...
9164,0,0,1,1,Customer Service Specialist Interpreting Servi...
9454,0,0,1,0,Customer Service Associate - Records Blank Nov...


In [166]:
y_test.unique()

array([0, 1], dtype=int64)

- X_rem and X_test have same features, and y_test and y_rem contains only 2 result.
- 2 Reminder sets and test stes have same number of rows.
- Now I have my reminder and test set ready.
---

# 3. NLP

## 3.1 Pre-process<a id="3.1"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

**Plan**
- Define function text_process()
- Transform X_rem and X_test


**Function text_process()**

**What does the function do**
- lowercase all the text
- Get rid of extra blank
- Take out of stopwords
- Take out puncuations
- Stemming
- Combine all the text back to a sentence

In [167]:
# init stemmer
stemmer = nltk.stem.PorterStemmer()
# make tokens
def text_process(text):
    non_pun=''.join([w for w in text.lower() if w not in string.punctuation])

    lst_words=non_pun.split(' ')
    #print(lst_words)
    # remove stop words and stem
    lst_out=[]
    for w in lst_words:
        if (w not in stopwords.words('english')) and (w != ""):
            stemmed_word = stemmer.stem(w)
            lst_out.append(stemmed_word)
    # combine list into sentence
            sentence = ' '.join(lst_out)
    return sentence

In [168]:
# test result
text_process(X_rem['text'][0])

'market intern market food52 weve creat groundbreak awardwin cook site support connect celebr home cook give everyth need one placew top editori busi engin team focus use technolog find new better way connect peopl around specif food interest offer superb highli curat inform food cook attract talent home cook contributor countri also publish wellknown profession like mario batali gwyneth paltrow danni meyer partnership whole food market random housefood52 name best food websit jame beard foundat iacp featur new york time npr pando daili techcrunch today showwer locat chelsea new york citi food52 fastgrow jame beard awardwin onlin food commun crowdsourc curat recip hub current interview full parttim unpaid intern work small team editor execut develop new york citi headquartersreproduc andor repackag exist food52 content number partner site huffington post yahoo buzzfe variou content manag systemsresearch blog websit provis food52 affili programassist daytoday affili program support scre

In [169]:
# compare with the orignal text
X_rem['text'][0]

"Marketing Intern Marketing We're Food52, and we've created a groundbreaking and award-winning cooking site. We support, connect, and celebrate home cooks, and give them everything they need in one place.We have a top editorial, business, and engineering team. We're focused on using technology to find new and better ways to connect people around their specific food interests, and to offer them superb, highly curated information about food and cooking. We attract the most talented home cooks and contributors in the country; we also publish well-known professionals like Mario Batali, Gwyneth Paltrow, and Danny Meyer. And we have partnerships with Whole Foods Market and Random House.Food52 has been named the best food website by the James Beard Foundation and IACP, and has been featured in the New York Times, NPR, Pando Daily, TechCrunch, and on the Today Show.We're located in Chelsea, in New York City. Food52, a fast-growing, James Beard Award-winning online food community and crowd-sour

**Note: This `text` pre-process process is quite time-consuming, if you don't want to run this, you can load from my pickle files**

**File list**
- X_rem_stem.pkl stemmed X_rem
- X_test_stem.pkl stemmed X_test

In [170]:
# X_rem pre-process 
X_rem_stem=X_rem.copy()
X_rem_stem['text']=X_rem_stem['text'].apply(text_process)

In [171]:
X_rem_stem.shape==X_rem.shape

True

In [221]:
X_rem_stem.shape

(12515, 5)

In [172]:
# X_test pre-process 
X_test_stem=X_test.copy()
X_test_stem['text']=X_test_stem['text'].apply(text_process)

In [173]:
X_test.shape==X_test_stem.shape

True

In [222]:
X_test_stem.shape

(5364, 5)

In [242]:
# dump stemmed X_rem
joblib.dump(X_rem_stem,'data_NN/X_rem_stem.pkl')

['data_NN/X_rem_stem.pkl']

In [243]:
# dump stemmed X_test
joblib.dump(X_test_stem,'data_NN/X_test_stem.pkl')

['data_NN/X_test_stem.pkl']

- X_rem and X_test has been transfered, moving forward I will keep NLP precoess using X_test_stem and X_rem_stem.
---

# 4. Word Embeddings

## 4.1 Word2Vec<a id="4.1"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>


- Word2Vec, you need to down load and unzip the pre_trained word vectors. load pretrained vectors: we will use [LexVec](https://github.com/alexandres/lexvec) and Wikipedia.
- This is a 300-dimensional embeddings.

In [69]:
# load model
model = gensim.models.KeyedVectors.load_word2vec_format('word_vectors/lexvec-wikipedia-word-vectors', binary=False)

In [70]:
def sentence2vec(text):
    """
    Embed a sentence by averaging the word vectors of the tokenized text. 
    Out-of-vocabulary words are replaced by the zero-vector.
    -----
    
    Input: text (string)
    Output: embedding vector (np.array)
    """
    tokenized = simple_preprocess(text)
    
    word_embeddings = [np.zeros(300)]
    for word in tokenized:
        # if the word is in the model then embed
        if word in model:
            vector = model[word]
        # add zeros for out-of-vocab words
        else:
            vector = np.zeros(300)
            
        word_embeddings.append(vector)
    
    # average the word vectors
    sentence_embedding = np.stack(word_embeddings).mean(axis=0)
    
    return sentence_embedding

In [43]:
# run test on function sentence2vec, funtion sentence2vec will vectorlize a chunk of words and make them into vectors.
sentence2vec(test)

array([-8.73303677e-03, -7.40721906e-03,  1.96714861e-02, -9.62654653e-03,
       -7.61171243e-03,  7.10465100e-02,  2.79783374e-03, -3.44043646e-02,
        2.67091537e-02,  3.47746526e-02,  3.64276721e-02,  4.86087006e-02,
       -5.14526681e-02,  5.95221453e-02,  3.72461983e-02, -7.26838076e-03,
       -2.58509070e-02,  1.05571504e-02,  1.63256722e-02,  2.45892592e-02,
       -2.23805102e-02, -1.03946277e-02,  3.08614728e-02,  2.79396841e-02,
       -3.21179997e-02,  2.59392796e-02,  6.13525187e-02,  6.66009756e-02,
        8.12814498e-02, -9.03423099e-03, -6.22478060e-02,  1.34164980e-02,
        1.39349112e-02, -1.21182504e-02,  6.12421055e-03,  4.17263240e-02,
        1.42471214e-02, -5.70988515e-04, -2.02543078e-02,  5.34754635e-03,
        4.08871904e-02,  6.40502865e-03, -4.73126478e-02, -7.77524050e-02,
       -1.58924088e-02, -3.13673809e-02,  3.28009921e-02, -1.21536515e-02,
       -3.01963932e-02, -5.86122256e-03,  2.21079427e-02,  2.29700792e-03,
       -3.03853055e-03,  

In [174]:
#  300-dimensional
sentence2vec(test).shape

(300,)

**column tranformer** (Not work)

In [175]:
numerical = list(X_rem_stem.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")

Numerical columns are: ['has_salary_range', 'telecommuting', 'has_company_logo', 'has_questions']


In [176]:
text = list(X_rem_stem.select_dtypes('object').columns)
print(f"text columns is: {text}")

text columns is: ['text']


In [114]:
# Create the column transformations list + columns to which to apply
col_transforms_nn = [('num','passthrough', numerical),
                ('sentence2vec', sentence2vec, 'text')]

In [118]:
# Create the column transformer
col_trans_nn = ColumnTransformer(col_transforms_nn,remainder='drop')

In [119]:
# fit
#pipeline.fit_transform(X_rem_stem)
col_trans_nn.fit(X_rem_stem)

TypeError: All estimators should implement fit and transform, or can be 'drop' or 'passthrough' specifiers. '<function sentence2vec at 0x000001A9F90160D0>' (type <class 'function'>) doesn't.

**Apply sentence2vec function**

**Plan**
- Apply sentence2vec() on X_rem_stem[text] combine transformed df with X_rem_stem[numerical]
- Apply sentence2vec() on X_rem_test[text] combine transformed df with X_rem_test[numerical]

**X_rem_stem**

In [223]:
X_rem_stem

,has_salary_range,telecommuting,has_company_logo,has_questions,text
4310,0,0,1,1,Creative Director - Art Blank Kettle is an ind...
11082,0,0,1,0,"CDL Driver Blank ABC Supply Co., Inc. is the n..."
14188,0,0,1,0,Data Analyst (Marketing) Blank Blank We are lo...
8280,0,0,1,1,Channel Representative Blank Intercom (# is a ...
13924,0,0,1,0,Programmatic Media Manager Media Since 1978Our...
...,...,...,...,...,...
1344,0,0,1,1,Sales Manager - Legal Services Sales Pardon Se...
7561,0,0,1,1,Texas Communication Assistant CSD Relay Why CS...
16552,1,0,1,0,"Director, Information Security Blank Blank Dir..."
14072,0,0,1,1,Sales & Support role Blank Blank We are a smal...


In [177]:
X_rem_word2vec_text = np.array([sentence2vec(x) for x in tqdm(X_rem_stem['text'])])

100%|██████████| 12515/12515 [00:14<00:00, 839.23it/s]


In [179]:
# keep the original row and 300d
X_rem_word2vec_text.shape

(12515, 300)

In [205]:
X_wv_text_df=pd.DataFrame(X_rem_word2vec_text).reset_index(drop=True)

In [207]:
print(f"The shape of X_wv_text_df is {X_wv_text_df.shape}")

The shape of X_wv_text_df is (12515, 300)


In [214]:
X_num_stem_df=X_rem_stem[numerical].reset_index(drop=True)
X_num_stem_df

,has_salary_range,telecommuting,has_company_logo,has_questions
0,0,0,1,1
1,0,0,1,0
2,0,0,1,0
3,0,0,1,1
4,0,0,1,0
...,...,...,...,...
12510,0,0,1,1
12511,0,0,1,1
12512,1,0,1,0
12513,0,0,1,1


In [215]:
# check if X_wv_text_df and X_num_stem_df has the same rows.
X_wv_text_df.shape[0]==X_num_stem_df.shape[0]

True

In [216]:
X_rem_wv_df=pd.concat([X_wv_text_df,X_num_stem_df],axis=1)

In [219]:
# transformed X_rem df
X_rem_wv_df

,0,1,2,3,4,5,6,7,8,9,...,294,295,296,297,298,299,has_salary_range,telecommuting,has_company_logo,has_questions
0,0.018585,-0.023172,-0.010314,-0.016143,-0.007745,0.057815,-0.017379,-0.040377,-0.000957,0.000053,...,0.008006,0.036052,0.008423,-0.009175,-0.021098,0.035679,0,0,1,1
1,-0.002319,-0.032277,-0.018184,0.013383,-0.011639,0.052527,-0.004319,-0.040372,-0.011319,0.011722,...,-0.001033,0.057966,0.003911,-0.014854,-0.004188,0.007001,0,0,1,0
2,0.021749,-0.010410,-0.004304,-0.010770,-0.004601,0.060114,-0.032992,-0.034776,0.037919,0.025008,...,0.001429,0.053172,-0.003019,-0.001675,-0.017215,0.016437,0,0,1,0
3,-0.000086,0.001106,-0.015206,0.015126,-0.001652,0.065113,-0.009476,-0.052275,0.023797,0.022757,...,0.017189,0.069419,-0.025309,-0.009836,-0.029361,0.012510,0,0,1,1
4,-0.012769,-0.024972,0.002534,-0.020615,-0.022524,0.060707,-0.008900,-0.023104,0.038573,-0.000873,...,0.014157,0.042389,0.013121,0.014692,-0.045533,0.008413,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12510,0.008069,-0.019125,0.018521,-0.033664,-0.011929,0.044832,-0.011822,-0.046514,0.037288,0.015168,...,0.005337,0.045893,0.003625,-0.017689,-0.034736,0.010429,0,0,1,1
12511,0.032365,-0.015908,0.015686,-0.016515,-0.014765,0.057625,-0.031313,-0.037653,0.003422,0.014710,...,0.033322,0.027682,-0.018315,-0.005030,-0.027586,0.015003,0,0,1,1
12512,0.003635,0.011399,-0.011378,-0.009706,-0.010376,0.073809,-0.028816,-0.046958,0.036731,0.016745,...,0.036082,0.042963,-0.008089,-0.053839,-0.019983,0.045565,1,0,1,0
12513,0.028538,-0.039625,-0.005739,0.010955,0.003517,0.064333,0.010148,-0.034340,0.009982,0.017702,...,-0.030521,0.031236,0.016533,-0.027138,-0.015386,-0.001245,0,0,1,1


In [239]:
X_rem_wv_df.shape

(12515, 304)

In [218]:
# there is no nan in the X_rem_wv_df
X_rem_wv_df.isna().sum().sum()

0

**X_rem_test**

In [220]:
X_test_stem

,has_salary_range,telecommuting,has_company_logo,has_questions,text
8222,1,0,1,0,ecommerc specialist databas manag home decor c...
1560,0,0,1,1,solut engin blank declara focus bring data lif...
17171,1,0,0,1,sale repres home improv blank blank look good ...
9164,0,0,1,1,custom servic specialist interpret servic rela...
9454,0,0,1,0,custom servic associ record blank novitex ente...
...,...,...,...,...,...
2611,0,0,1,0,stock manag blank blank execut flow merchandis...
11822,0,0,1,0,cabl tech night shift blank tel tech network v...
10961,0,0,1,1,advisori consult associ blank green street adv...
13509,0,0,1,0,dish washer 3040 hr per week blank belfri inn ...


In [224]:
X_test_word2vec_text = np.array([sentence2vec(x) for x in tqdm(X_test_stem['text'])])

100%|██████████| 5364/5364 [00:06<00:00, 785.70it/s]


In [225]:
# keep the original row and 300d
X_test_word2vec_text.shape

(5364, 300)

In [226]:
X_test_wv_text_df=pd.DataFrame(X_test_word2vec_text).reset_index(drop=True)

In [229]:
# check if transformed df shape is not changed 
X_test_wv_text_df.shape==X_test_word2vec_text.shape

True

In [231]:
X_test_num_stem_df=X_test_stem[numerical].reset_index(drop=True)

In [234]:
# check if transformed df shape is not changed 
X_test_num_stem_df.shape==X_test_stem[numerical].shape

True

In [235]:
# test set: numerical features and text features has same rows 
X_test_num_stem_df.shape[0]==X_test_wv_text_df.shape[0]

True

In [236]:
# combine test set
X_test_wv_df=pd.concat([X_test_wv_text_df,X_test_num_stem_df],axis=1)

In [237]:
X_test_wv_df

,0,1,2,3,4,5,6,7,8,9,...,294,295,296,297,298,299,has_salary_range,telecommuting,has_company_logo,has_questions
0,0.018789,-0.022035,-0.013473,0.011040,0.001409,0.095111,0.007145,-0.054297,0.044798,0.040261,...,0.007426,0.037257,-0.023748,-0.006802,-0.024362,0.016189,1,0,1,0
1,0.010367,-0.050015,-0.010797,-0.007565,0.000990,0.048667,-0.014224,-0.055378,0.009178,0.042945,...,0.020617,0.053427,-0.012712,-0.027088,-0.010420,0.032028,0,0,1,1
2,-0.000405,-0.014026,-0.019699,0.005127,-0.017995,0.043106,-0.011168,-0.019834,-0.002494,-0.000975,...,-0.045438,0.080359,-0.031152,-0.004918,0.009107,0.053894,1,0,0,1
3,0.025157,-0.009937,0.010194,-0.006731,0.005550,0.053188,-0.027584,-0.029183,0.002354,0.012742,...,0.017930,0.038814,-0.025250,-0.016356,-0.011946,0.024874,0,0,1,1
4,0.015688,-0.003842,-0.021589,-0.003563,-0.012856,0.071822,-0.014360,-0.032929,0.020752,0.014072,...,0.006578,0.048972,-0.019309,-0.004729,-0.018557,0.011441,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5359,0.009623,-0.038822,0.007203,-0.003796,-0.022963,0.027172,0.011261,-0.080157,0.034272,-0.006636,...,-0.026421,0.048347,-0.001478,-0.004622,-0.026346,-0.001985,0,0,1,0
5360,0.005633,-0.036866,-0.004559,0.013722,-0.038692,0.063361,-0.004791,-0.048095,0.006991,0.001913,...,0.020413,0.055088,0.011159,-0.020813,-0.008464,0.008605,0,0,1,0
5361,-0.004940,-0.017801,0.012386,-0.027455,0.022083,0.048710,-0.009394,-0.057615,0.042862,-0.009790,...,0.031981,0.064530,0.005916,-0.020952,-0.002602,0.030358,0,0,1,1
5362,-0.001808,-0.074149,-0.000581,-0.003270,-0.002644,0.047228,0.013216,-0.050476,0.055103,0.000468,...,-0.022682,0.093179,-0.026688,-0.038551,0.017785,-0.015965,0,0,1,0


In [238]:
X_test_wv_df.shape

(5364, 304)

In [240]:
X_test_wv_df.isna().sum().sum()

0

In [241]:
print(f'The shape of transformed (word2vec) reminder is {X_rem_wv_df.shape},shape of transformed test set is {X_test_wv_df.shape} ')

The shape of transformed (word2vec) reminder is (12515, 304),shape of transformed test set is (5364, 304) 


## 4.2 GloVe<a id="4.2"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>